<a href="https://colab.research.google.com/github/truongcan0512/Image-analysis/blob/main/Lab2_Color_Moment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/leedsbutterfly_dataset_v1.0.zip

In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [6]:
# Buoc 1: Trich xuat object pixel
"""
Ta se chon ra 30 images de tinh color centers
Y tuong: 
- Voi moi object, ta tim ra nhung pixels khac khong bang cach tim
trong mask.
- Sau do, ung moi moi toa do x,y trong mask, ta match voi image -> obj pixels
"""
List_images = os.listdir("/content/leedsbutterfly/images") #random
num_images = len(List_images)
OBJ_pixels = np.zeros((1,3)) 

# Chon 30 images de tinh color centers

for i in range(0,30):
  imBGR = cv2.imread("/content/leedsbutterfly/images/" + List_images[i]) #Đọc ảnh, chỉ cần đưa tên đường dẫn ảnh vào -> ảnh BRG
  imRGB = cv2.cvtColor(imBGR, cv2.COLOR_BGR2RGB) #Chuyển ảnh về dạng RGB
  imRGB = imRGB/255 # Chuan hoa ve [0;1]

  seg_path = "/content/leedsbutterfly/segmentations/" + List_images[i].replace(".png","_seg0.png") # Cac goi file segmentation bang cach ".png"-->"_seg0.png"
  mask = cv2.imread(seg_path)
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

  [x,y]=np.nonzero(mask[:,:,0]) # Ham nonzeros tra ve index cua cac phan tu khac 0 trong ma tran
  obj=imRGB[x,y,:] # Lay pixel cua object
  OBJ_pixels = np.concatenate((OBJ_pixels, obj), axis=0) # Noi cac obj theo hang

In [28]:
# Buoc 2: Quantization
"""
Muc dich: Dua anh ve chi con 16 mau chinh.
Y tuong:
- Ta chon ra mot tap con bao gom 10.000 pixels trong tong so OBJ_pixels
- Sau do, ta su dung clustering de phan cum du lieu ta thu duoc cac pixel
centers, ta thay cac pixels bang cac pixel centers --> quantization
"""

sub_image_array = shuffle(OBJ_pixels,n_samples=10000) # Chon ngau nhien

# Build model using KMeans with 16 clusters
color_quantization_model = KMeans(n_clusters=16) # Build model
color_quantization_model.fit(sub_image_array) # Fit model

# Quantization image
quantization_images = color_quantization_model.cluster_centers_[color_quantization_model.predict(OBJ_pixels)] #Chon ra 16 mau chinh cho 30 images
#np.shape(quantization_images)
#quantization_image = np.reshape(quantization_images,(768,1024,3)) #khoi phuc lai anh voi kich co ban dau
#plt.imshow(quantization_image)
#plt.show()

In [58]:
import scipy
# Buoc 3: Trich xuat dac trung cho anh, dua ve dang vector cua ML
"""
Muc dich: Voi moi anh, ta se xem xet so lan xuat hien cua moi centers
Y tuong:
- Su dung ham histogram trong thu vien numpy
"""
from scipy.stats import skew

X=[] # feature vector includes E, sigma, skew for each image
label=[] # label
for i in range(0,num_images):
  imBGR = cv2.imread("/content/leedsbutterfly/images/" + List_images[i]) #Đọc ảnh, chỉ cần đưa tên đường dẫn ảnh vào -> ảnh BRG
  imRGB = cv2.cvtColor(imBGR, cv2.COLOR_BGR2RGB) #Chuyển ảnh về dạng RGB
  imRGB = imRGB/255

  seg_path = "/content/leedsbutterfly/segmentations/" + List_images[i].replace(".png","_seg0.png") # Cac goi file segmentation bang cach ".png"-->"_seg0.png"
  mask = cv2.imread(seg_path)
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

  [x,y]=np.nonzero(mask[:,:,0]) # Ham nonzeros tra ve index cua cac phan tu khac 0 trong ma tran
  obj=imRGB[x,y,:] # Lay pixel cua object
  quantization_obj = color_quantization_model.cluster_centers_[color_quantization_model.predict(obj)]
  color_id = color_quantization_model.predict(obj) #ma tran color_id bieu dien cac label cho moi pixel cua object

  # Color Moment
  E_red = np.mean(quantization_obj[:,0])
  sigma_red = np.std(quantization_obj[:,0])
  skew_red = scipy.stats.skew(quantization_obj[:,0])

  E_green = np.mean(quantization_obj[:,1])
  sigma_green = np.std(quantization_obj[:,1])
  skew_green = scipy.stats.skew(quantization_obj[:,1])

  E_blue = np.mean(quantization_obj[:,2])
  sigma_blue = np.std(quantization_obj[:,2])
  skew_blue = scipy.stats.skew(quantization_obj[:,2])


  sample_image = [E_red,E_green,E_blue,sigma_red,sigma_green,sigma_blue,skew_red,skew_green,skew_blue ]
  X.append(sample_image)
  label.append(int(List_images[i][0:3]))

In [59]:
y = np.array(label)
np.shape(X)

(832, 9)

In [60]:
# Buoc 4: Tao training/test

# Loai bo 30 phan tu dau tien 
X = np.array(X)
y = np.array(label)
X_data = X[30:]
y_data = y[30:]

# Lay moi class 15 images lam test
X_test = []
y_test = []
X_train = []
y_train = []
count = 0
for i in range(1,11):
  tmp = 0
  for j in range(0,np.shape(y_data)[0]):
    if(y_data[j] == i and tmp < 15):
      X_test.append(X_data[j])
      y_test.append(y_data[j])
      tmp = tmp+1
    else:
      if(count < np.shape(y_data)[0]):
        y_train.append(y_data[j])
        X_train.append(X_data[j])
      count = count + 1

X_test = np.array(X_test)
y_test = np.array(y_test)
X_train = np.array(X_train)
y_train = np.array(y_train)

In [61]:
knn_model=KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train,y_train)
knn_test=knn_model.predict(X_test)
print("Acc of KNN: ", accuracy_score(y_test, knn_test))

svm_model = SVC(C=999999, kernel="rbf", gamma=2, probability=True)
svm_model.fit(X_train, y_train)
print("Acc of SVM: ", svm_model.score(X_test, y_test))

Acc of KNN:  0.7666666666666667
Acc of SVM:  0.98
